# Preparo da massa de áudios em formato WAV

### Referências

https://medium.com/@keur.plkar/audio-data-augmentation-in-python-a91600613e47

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import librosa
import librosa.display
import soundfile as sf
import matplotlib.pyplot as plt
import IPython.display as ipd

In [2]:
# diretório de entrada
DIR_ENTRADA = 'mp3'

# diretório de saída
DIR_SAIDA = 'wavbal5'

# duração de cada trecho de áudio
DURACAO_TRECHO = 5 # segundos
SAMPLE_RATE = 16000 # 16 kHz

# calcular a quantidade de pontos na onda
PONTOS_ONDA = SAMPLE_RATE * DURACAO_TRECHO # 16 kHz * 5 s = 80000

## Verificar espécies disponíveis

In [3]:
ARQUIVO_ESPECIES = 'especies-oiapoque.txt'

especies = []
with open(ARQUIVO_ESPECIES, 'r') as f:
    especies = [especie.rstrip().lower().replace(' ', '_') for especie in f.readlines()]
especies

['ramphocelus_carbo',
 'pitangus_sulphuratus',
 'querula_purpurata',
 'cacicus_cela',
 'pteroglossus_aracari',
 'tangara_episcopus',
 'dacnis_cayana',
 'ramphastos_tucanus',
 'tangara_palmarum',
 'pionus_fuscus',
 'trogon_melanurus',
 'glyphorynchus_spirurus',
 'cyclarhis_gujanensis',
 'lipaugus_vociferans',
 'pionites_melanocephalus',
 'aramides_cajaneus',
 'ramphastos_vitellinus',
 'cathartes_aura',
 'turdus_leucomelas',
 'amazona_farinosa',
 'thamnophilus_punctatus',
 'thamnomanes_ardesiacus',
 'euphonia_violacea',
 'pithys_albifrons',
 'tachycineta_albiventer',
 'mionectes_macconnelli',
 'hypocnemis_cantator',
 'leptotila_rufaxilla',
 'dacnis_lineata',
 'tyrannus_melancholicus',
 'trogon_viridis',
 'manacus_manacus',
 'geotrygon_montana',
 'piaya_cayana',
 'myiopagis_gaimardii',
 'chloroceryle_americana',
 'trogon_violaceus',
 'bucco_capensis',
 'myrmotherula_surinamensis',
 'myiarchus_ferox',
 'phaethornis_sp',
 'thamnophilus_murinus',
 'dryocopus_lineatus',
 'formicarius_colma',


In [4]:
arquivos = []
especies_disponiveis = []

for especie in especies:
    dir_especie = os.path.join(DIR_ENTRADA, especie)
    if os.path.isdir(dir_especie):
        arquivos.append(len(os.listdir(dir_especie)) - 1)
        especies_disponiveis.append(especie)

especies = especies_disponiveis

In [5]:
pd.set_option('display.max_rows', 100)

df = pd.DataFrame({
    'especie': especies,
    'arquivos_mp3': arquivos
})

df.sort_values('especie')

,especie,arquivos_mp3
19,amazona_farinosa,46
15,aramides_cajaneus,37
54,attila_spadiceus,250
62,automolus_infuscatus,120
37,bucco_capensis,52
3,cacicus_cela,293
69,campephilus_rubricollis,120
17,cathartes_aura,7
50,celeus_undatus,25
53,ceratopipra_erythrocephala,147


## Recriar diretórios de saída

In [6]:
# excluir diretórios de saída

import shutil
shutil.rmtree(DIR_SAIDA)
print(DIR_SAIDA)

wavbal5


In [7]:
# criar diretórios de saída

if not os.path.isdir(DIR_SAIDA):
    os.mkdir(DIR_SAIDA)
print(DIR_SAIDA)

for especie in especies:
    dir_especie = os.path.join(DIR_SAIDA, especie)
    if not os.path.isdir(dir_especie):
        os.mkdir(dir_especie)
    print(dir_especie)

wavbal5
wavbal5/ramphocelus_carbo
wavbal5/pitangus_sulphuratus
wavbal5/querula_purpurata
wavbal5/cacicus_cela
wavbal5/pteroglossus_aracari
wavbal5/tangara_episcopus
wavbal5/dacnis_cayana
wavbal5/ramphastos_tucanus
wavbal5/tangara_palmarum
wavbal5/pionus_fuscus
wavbal5/trogon_melanurus
wavbal5/glyphorynchus_spirurus
wavbal5/cyclarhis_gujanensis
wavbal5/lipaugus_vociferans
wavbal5/pionites_melanocephalus
wavbal5/aramides_cajaneus
wavbal5/ramphastos_vitellinus
wavbal5/cathartes_aura
wavbal5/turdus_leucomelas
wavbal5/amazona_farinosa
wavbal5/thamnophilus_punctatus
wavbal5/thamnomanes_ardesiacus
wavbal5/euphonia_violacea
wavbal5/pithys_albifrons
wavbal5/tachycineta_albiventer
wavbal5/mionectes_macconnelli
wavbal5/hypocnemis_cantator
wavbal5/leptotila_rufaxilla
wavbal5/dacnis_lineata
wavbal5/tyrannus_melancholicus
wavbal5/trogon_viridis
wavbal5/manacus_manacus
wavbal5/geotrygon_montana
wavbal5/piaya_cayana
wavbal5/myiopagis_gaimardii
wavbal5/chloroceryle_americana
wavbal5/trogon_violaceus
wa

## Processar arquivos de áudio

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time

for especie in especies:
    print("especie:", especie)
    qtde_arquivos = 0

#    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie))[:LIMITE_ARQUIVOS_ENTRADA]:
#    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie))[:20]:
    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie)):

        # montar nome do arquivo de entrada
        entrada = os.path.join(DIR_ENTRADA, especie, arquivo)
        if not entrada.endswith('.mp3'):
            continue
        #print("entrada:", entrada)

        prefixo_saida = os.path.join(DIR_SAIDA, especie, arquivo.replace('.mp3', '-'))
        #print("prefixo_saida:", prefixo_saida)
        
        # carregar áudio original em formato MP3
        y, sr = librosa.load(entrada, sr=SAMPLE_RATE, mono=True)

        # remover silêncio nas extremidades do áudio
        yt, index = librosa.effects.trim(y)
        duracao = yt.shape[0] / sr
        
        # calcular o número de cortes a serem efetuados no áudio
        numero_cortes = round(duracao / DURACAO_TRECHO)
        if numero_cortes == 0:
            continue
        
        for corte in np.arange(1, numero_cortes + 1):
            saida = prefixo_saida + "0-" + str(corte) + ".wav"
            
            # calcular trechos de início e fim no áudio
            inicio = PONTOS_ONDA * (corte - 1)
            termino = PONTOS_ONDA * corte
            ytc = yt[inicio:termino]
            
            # gravar arquivo de áudio em formato WAV
            sf.write(saida, ytc, sr, format='wav', subtype='PCM_16')
            qtde_arquivos += 1

    print("-> qtde_arquivos:", qtde_arquivos, "\n")

especie: ramphocelus_carbo
-> qtde_arquivos: 1220 

especie: pitangus_sulphuratus


## Calcular quantidade de arquivos gerados

In [ ]:
arquivos = []
for especie in especies:
    dir_especie = os.path.join(DIR_SAIDA, especie)
    arquivos.append(len(os.listdir(dir_especie)))
#arquivos

In [ ]:
df['arquivos_wav'] = arquivos
maior_qtde = max(df['arquivos_wav'])
df['diferenca'] = maior_qtde - df['arquivos_wav']
df['multiplicacao'] = df[['diferenca', 'arquivos_mp3']].apply(
    lambda x: np.round(x[0] / x[1], 0), axis=1)
df.sort_values('especie')

## Gerar arquivos adicionais com deslocamento

In [ ]:
%%time

#for especie in ['theristicus_caudatus']:
for especie in especies:
    print("especie:", especie)
    qtde_arquivos = 0
        
    # multiplicação necessária para a espécie
    multiplicacao = int(df[df.especie == especie]['multiplicacao'].values[0])
    print("-> multiplicacao:", multiplicacao)
    if multiplicacao == 0:
        continue

    diferenca = int(df[df.especie == especie]['diferenca'].values[0])
    print("-> diferenca:", diferenca)

#    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie))[:1]:
#    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie))[:20]:
    for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie)):

        # sair do laço se atingir a quantidade necessária
        if qtde_arquivos >= diferenca:
            break
            
        # montar nome do arquivo de entrada
        entrada = os.path.join(DIR_ENTRADA, especie, arquivo)
        if not entrada.endswith('.mp3'):
            continue
        #print("entrada:", entrada)

        prefixo_saida = os.path.join(DIR_SAIDA, especie, arquivo.replace('.mp3', '-'))
        #print("prefixo_saida:", prefixo_saida)
        
        # carregar áudio original em formato MP3
        y, sr = librosa.load(entrada, sr=SAMPLE_RATE, mono=True)
        numero_pontos = y.shape[0]
        #print("-> numero_pontos:", numero_pontos)
        duracao = y.shape[0] / sr
        #print("-> duracao:", duracao, "seg")

        # calcular o número de cortes a serem efetuados no áudio
        numero_cortes = round(duracao / DURACAO_TRECHO)
        #print("-> numero_cortes:", numero_cortes)
        if numero_cortes == 0:
            continue
        
        # calcular o número necessário de variações
        numero_variacoes = round(multiplicacao / numero_cortes)
        #print("-> numero_variacoes:", numero_variacoes)
        
        for variacao in np.arange(1, numero_variacoes + 1):
            
            # sair do laço se atingir a quantidade necessária
            if qtde_arquivos >= diferenca:
                break

            # deslocar o sinal (roll)
            tamanho_roll = variacao * int(PONTOS_ONDA / (variacao + 1))
            #print("--> tamanho_roll:", tamanho_roll)
            yr = np.roll(y, tamanho_roll)
            
            # adicionar ruído ao sinal
            yrs = yr + 0.009 * np.random.normal(0, 1, len(yr))
            
            for corte in np.arange(1, numero_cortes + 1):

                # sair do laço se atingir a quantidade necessária
                if qtde_arquivos >= diferenca:
                    break

                saida = prefixo_saida + str(variacao) + "-" + str(corte) + ".wav"
                    
                # calcular trechos de início e fim no áudio
                inicio = PONTOS_ONDA * (corte - 1)
                termino = PONTOS_ONDA * corte
                yrc = yrs[inicio:termino]
                
                duracao = yrc.shape[0] / sr
                if (duracao / DURACAO_TRECHO < 0.3):
                    break

                # gravar arquivo de áudio em formato WAV
                sf.write(saida, yrc, sr, format='wav', subtype='PCM_16')
                qtde_arquivos += 1
                
    print("-> qtde_arquivos:", qtde_arquivos, "\n")

## Calcular quantidade de arquivos gerados ao final

In [ ]:
arquivos = []
for especie in especies:
    dir_especie = os.path.join(DIR_SAIDA, especie)
    arquivos.append(len(os.listdir(dir_especie)))
#arquivos

In [ ]:
df['arquivos_wav2'] = arquivos
maior_qtde2 = max(df['arquivos_wav2'])
df['diferenca2'] = maior_qtde2 - df['arquivos_wav2']
df.sort_values('especie')